<a href="https://colab.research.google.com/github/Hatakun/Prog2_11th/blob/main/work3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2 , sys
from google.colab.patches import cv2_imshow
print('画像のファイルの名前を拡張子付きで入力してください')
#画像の名前を取得
img1_name = input('名前を入力してください')
img2_name = input('名前を入力してください')

#画像の読み込み
img1 = cv2.imread(img1_name)
img2 = cv2.imread(img2_name)


if img1 is None:
  sys.exit(f'エラー：ファイルを開けませんでした{img1_name}')

if img2 is None:
  sys.exit(f'エラー：ファイルを開けませんでした{img2_name}')

if img1.shape != img2.shape:
  c1 = img1.shape[1]/img1.shape[0]
  c2 = img2.shape[1]/img2.shape[0]

  if c1 != c2:
    sys.print('画像の縦横比が違うので比較できません')

